# 금공프3 Final

20249433 MFE 최재필

In [249]:
from functools import reduce

import numpy as np
import pandas as pd

## 1. Mean-Variance Optimization

In [134]:
# 1

In [135]:
asset = pd.read_csv('price.csv')

In [136]:
asset.set_index('date', inplace=True)

In [172]:
asset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 565 entries, 2018-05-23 to 2020-09-04
Columns: 2203 entries, X005930 to X158310
dtypes: float64(1722), int64(481)
memory usage: 9.5+ MB


In [175]:
asset.index

Index(['2018-05-23', '2018-05-24', '2018-05-25', '2018-05-28', '2018-05-29',
       '2018-05-30', '2018-05-31', '2018-06-01', '2018-06-04', '2018-06-05',
       ...
       '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28',
       '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04'],
      dtype='object', name='date', length=565)

In [152]:
small_asset = asset.iloc[:10, :5].copy()
small_asset.iloc[:7, 0] = np.nan
small_asset.iloc[3:5, 1] = np.nan
small_asset.iloc[-7:, 2] = np.nan

small_asset # 극단적인 케이스 가정하여 만듦

,X005930,X000660,X051910,X207940
date,,,,
2018-05-23,NaN,95300.0,343500.0,399500.0
2018-05-24,NaN,94600.0,345000.0,418000.0
2018-05-25,NaN,95200.0,352500.0,430500.0
2018-05-28,NaN,NaN,NaN,429000.0
2018-05-29,NaN,NaN,NaN,433000.0
2018-05-30,NaN,95000.0,NaN,425500.0
2018-05-31,NaN,93400.0,NaN,435000.0
2018-06-01,51300.0,91400.0,NaN,451500.0
2018-06-04,51100.0,89800.0,NaN,419000.0


In [ ]:
small_asset_2d = small_asset.to_numpy()
small_asset_2d

# TODO: quant research 시 좋은 approach. util 로 빼서 case별로 small 만들어주는 툴 있으면 좋음. 


array([[    nan,  95300., 343500., 399500.],
       [    nan,  94600., 345000., 418000.],
       [    nan,  95200., 352500., 430500.],
       [    nan,     nan,     nan, 429000.],
       [    nan,     nan,     nan, 433000.],
       [    nan,  95000.,     nan, 425500.],
       [    nan,  93400.,     nan, 435000.],
       [ 51300.,  91400.,     nan, 451500.],
       [ 51100.,  89800.,     nan, 419000.],
       [ 51300.,  91000.,     nan, 421000.]])

In [ ]:
small_asset_2d.shape

(10, 4)

In [153]:
asset_2d = asset.to_numpy()
asset_2d

array([[5.180e+04, 9.530e+04, 3.435e+05, ...,       nan, 5.079e+03,
        1.025e+04],
       [5.140e+04, 9.460e+04, 3.450e+05, ...,       nan, 5.005e+03,
        9.300e+03],
       [5.270e+04, 9.520e+04, 3.525e+05, ...,       nan, 4.870e+03,
        8.920e+03],
       ...,
       [5.440e+04, 7.550e+04, 7.420e+05, ..., 2.015e+03, 4.980e+02,
        5.050e+02],
       [5.640e+04, 7.870e+04, 7.680e+05, ..., 2.010e+03, 4.980e+02,
        5.050e+02],
       [5.560e+04, 7.870e+04, 7.430e+05, ...,       nan,       nan,
              nan]])

In [159]:
small_X = np.log(small_asset_2d[1:]/small_asset_2d[:-1])
small_X

array([[        nan, -0.00737233,  0.00435731,  0.04526767],
       [        nan,  0.00632247,  0.02150621,  0.02946589],
       [        nan,         nan,         nan, -0.0034904 ],
       [        nan,         nan,         nan,  0.00928081],
       [        nan,         nan,         nan, -0.01747278],
       [        nan, -0.01698555,         nan,  0.02208108],
       [        nan, -0.02164587,         nan,  0.03722934],
       [-0.00390625, -0.0176605 ,         nan, -0.07470445],
       [ 0.00390625,  0.01327453,         nan,  0.00476191]])

In [160]:
small_X.shape

(9, 4)

In [161]:
X = np.log(asset_2d[1:]/asset_2d[:-1]) # 수익률 (수익률은 log 차분 수익률로 계산)
X.shape

(564, 2203)

In [162]:
# to address nan values, use pandas for covariance matrix

small_X_df = pd.DataFrame(small_X, columns=small_asset.columns)

In [165]:
small_X_df.cov(min_periods=2)

,X005930,X000660,X051910,X207940
X005930,0.000031,0.000121,NaN,0.000310
X000660,0.000121,0.000203,0.000117,0.000105
X051910,NaN,0.000117,0.000147,-0.000135
X207940,0.000310,0.000105,-0.000135,0.001312


In [168]:
X_df = pd.DataFrame(X, columns=asset.columns)

In [166]:
small_Q = small_X_df.cov(min_periods=2).to_numpy()
small_Q

array([[ 3.05176557e-05,  1.20840132e-04,             nan,
         3.10415890e-04],
       [ 1.20840132e-04,  2.03165993e-04,  1.17425380e-04,
         1.04994847e-04],
       [            nan,  1.17425380e-04,  1.47042383e-04,
        -1.35491537e-04],
       [ 3.10415890e-04,  1.04994847e-04, -1.35491537e-04,
         1.31159219e-03]])

In [167]:
small_Q.shape

(4, 4)

In [169]:
Q = X_df.cov(min_periods=2).to_numpy()

In [171]:
np.isnan(Q).sum()  

277888

In [50]:
small_r = np.mean(small_X, axis=0)
small_r = small_r.reshape(-1, 1)
small_r.shape


(5, 1)

In [51]:
r = np.mean(X, axis=0) # 기대값 (수익률 평균)
r = r.reshape(-1, 1)

In [52]:
small_l = np.ones(small_r.shape)
small_l

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [54]:
small_l.shape

(5, 1)

In [57]:
l = np.ones(r.shape) # 1 벡터

In [71]:
small_Q_l_r = np.hstack([small_Q, small_l, small_r])
small_Q_l_r

array([[ 3.90341297e-04, -5.69506461e-05,  3.32604512e-04,
         2.73799928e-04,  2.91567602e-04,  1.00000000e+00,
        -1.07771079e-03],
       [-5.69506461e-05,  1.46447778e-04, -1.69814194e-04,
         4.26802830e-05, -7.84137101e-05,  1.00000000e+00,
        -5.13003379e-03],
       [ 3.32604512e-04, -1.69814194e-04,  1.04134235e-03,
        -5.28644749e-04,  1.72710965e-04,  1.00000000e+00,
         6.28802059e-03],
       [ 2.73799928e-04,  4.26802830e-05, -5.28644749e-04,
         1.31159219e-03,  2.84438052e-04,  1.00000000e+00,
         5.82434094e-03],
       [ 2.91567602e-04, -7.84137101e-05,  1.72710965e-04,
         2.84438052e-04,  2.57507560e-04,  1.00000000e+00,
        -1.63655774e-03]])

In [72]:
small_Q_l_r.shape

(5, 7)

In [64]:
Q_l_r = np.hstack([Q, l, r])

In [68]:
small_zero = np.zeros(small_l.shape)
small_zero

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [69]:
zero = np.zeros(l.shape)

In [75]:
small_l_0_0 = np.hstack([small_l.T, [[0]], [[0]]])
small_l_0_0

array([[1., 1., 1., 1., 1., 0., 0.]])

In [ ]:
small_l_0_0.shape

(1, 7)

In [77]:
l_0_0 = np.hstack([l.T, [[0]], [[0]]])

In [80]:
small_r_0_0 = np.hstack([small_r.T, [[0]], [[0]]])
small_r_0_0

array([[-0.00107771, -0.00513003,  0.00628802,  0.00582434, -0.00163656,
         0.        ,  0.        ]])

In [81]:
small_r_0_0.shape

(1, 7)

In [83]:
r_0_0 = np.hstack([r.T, [[0]], [[0]]]) 

In [84]:
small_L = np.vstack([small_Q_l_r, small_l_0_0, small_r_0_0])
small_L

array([[ 3.90341297e-04, -5.69506461e-05,  3.32604512e-04,
         2.73799928e-04,  2.91567602e-04,  1.00000000e+00,
        -1.07771079e-03],
       [-5.69506461e-05,  1.46447778e-04, -1.69814194e-04,
         4.26802830e-05, -7.84137101e-05,  1.00000000e+00,
        -5.13003379e-03],
       [ 3.32604512e-04, -1.69814194e-04,  1.04134235e-03,
        -5.28644749e-04,  1.72710965e-04,  1.00000000e+00,
         6.28802059e-03],
       [ 2.73799928e-04,  4.26802830e-05, -5.28644749e-04,
         1.31159219e-03,  2.84438052e-04,  1.00000000e+00,
         5.82434094e-03],
       [ 2.91567602e-04, -7.84137101e-05,  1.72710965e-04,
         2.84438052e-04,  2.57507560e-04,  1.00000000e+00,
        -1.63655774e-03],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.07771079e-03, -5.13003379e-03,  6.28802059e-03,
         5.82434094e-03, -1.63655774e-03,  0.00000000e+00,
         0.0000000

In [85]:
small_L.shape

(7, 7)

In [86]:
L = np.vstack([Q_l_r, l_0_0, r_0_0])

In [92]:
MU = 0.01 # 주어진 포트폴리오 수익률

In [93]:
np.nanmean(r)

-6.0143105726468836e-05

In [100]:
small_zero = np.zeros(small_l.shape)
small_zero

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [101]:
zero = np.zeros(l.shape)

In [102]:
small_zero_l_mu = np.vstack([small_zero, [[0]], [[MU]]])
small_zero_l_mu

array([[0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.01]])

In [103]:
zero_l_mu = np.vstack([zero, [[0]], [[MU]]])

In [104]:
small_L_inv = np.linalg.inv(small_L)
small_L_inv.shape

(7, 7)

In [105]:
L_inv = np.linalg.inv(L)

In [107]:
small_w_lmda1_lmda2 = small_L_inv @ small_zero_l_mu
small_w_lmda1_lmda2

array([[-8.26198516e-01],
       [-2.01322689e-01],
       [ 6.78935765e-01],
       [ 5.86809051e-01],
       [-2.38223610e-01],
       [-2.19478506e-05],
       [-1.48042550e-02]])

In [108]:
w_lmda1_lmda2 = L_inv @ zero_l_mu

In [111]:
small_w = small_w_lmda1_lmda2[:-2]
lmda1 = small_w_lmda1_lmda2[-2]
lmda2 = small_w_lmda1_lmda2[-1]

In [112]:
w = w_lmda1_lmda2[:-2]

In [110]:
lmda1 = w_lmda1_lmda2[-2]
lmda2 = w_lmda1_lmda2[-1]

In [115]:
small_var = small_w.T @ small_Q @ small_w
small_var

array([[0.00014804]])

In [117]:
var = w.T @ Q @ w
var

array([[nan]])

In [4]:
# 2. 

w = None # 포트폴리오 비중

mu = None # 포트폴리오 P의 기대수익률
var = None # 포트폴리오 P의 분산
sigma = np.sqrt(var) # 포트폴리오 P의 표준편차

TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [5]:
# 3. 

objective = None # 목적함수

In [6]:
# 4. 

Lagrangian = None # 라그랑지안 편미분 행렬

### (1) `MVportfolio` 

In [ ]:
def MVportfolio(asset, mu_p):
    price_2d = price.to_numpy()

    # 1. 

    X = None # 수익률 (수익률은 log 차분 수익률로 계산)
    Q = None # 공분산 행렬
    r = None # 기대값 (수익률 평균)

    asset = None # k개의 자산 별 가격을 m일 동안 수집한 mxk 의 dataframe
    w = None # k개의 자산 별 가중치
    var = None # 포트폴리오 P의 분산

    return asset, w, var

### (2) Efficient Frontier

In [7]:
it = pd.read_csv('it.csv')
it

,Date,AAPL,GOOG,MSFT,IBM,T
0,2018-01-02,169.714086,1065.00,84.484767,147.537554,36.382354
1,2018-01-03,169.684530,1082.48,84.877948,151.593043,35.542180
2,2018-01-04,170.472706,1086.40,85.624992,154.663355,35.872586
3,2018-01-05,172.413591,1102.23,86.686581,155.418977,36.004748
4,2018-01-08,171.773197,1106.94,86.775047,156.356330,36.108590
...,...,...,...,...,...,...
227,2018-11-26,174.620000,1048.62,106.470000,119.560000,29.950000
228,2018-11-27,174.240000,1044.41,107.140000,120.030000,30.440000
229,2018-11-28,180.940000,1086.23,111.120000,123.000000,30.740000
230,2018-11-29,179.550000,1088.30,110.190000,121.480000,30.570000


## 2. Momentum

## 3. Monte-Carlo Simulation